In [ ]:
import pandas as pd
from collections import deque
import speech_recognition as sr
import pyttsx3
import socket
import threading
import time
from pinpong.board import Board
from pinpong.extension.unihiker import accelerometer, gyroscope
import math

UDP_IP = "192.168.137.13"  # ESP8266 IP
UDP_PORT = 4215
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
engine = pyttsx3.init()

# Fall detection thresholds 
ACCEL_THRESHOLD = 1.0  
GYRO_THRESHOLD = 0.3 

def detect_motion():
    while True:
        acc_x = accelerometer.get_x()
        acc_y = accelerometer.get_y()
        acc_z = accelerometer.get_z()
        gyr_x = gyroscope.get_x()
        gyr_y = gyroscope.get_y()
        gyr_z = gyroscope.get_z()

        acc_magnitude = math.sqrt(acc_x**2 + acc_y**2 + acc_z**2)
        gyro_magnitude = math.sqrt(gyr_x**2 + gyr_y**2 + gyr_z**2)

        if acc_magnitude > ACCEL_THRESHOLD and gyro_magnitude > GYRO_THRESHOLD:
            fall_alert = "FD"
            sock.sendto(fall_alert.encode(), (UDP_IP, UDP_PORT))
            print("Fall detected! Sending alert...")

def speak(text):
    """Function to output audio for the given text."""
    print(text) 
    engine.say(text)
    engine.runAndWait()

def Recognize_Speech(location):
    """Function to recognize speech input using microphone."""
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    with microphone as source:
        if location == 'current':
            speak('Provide current location:')
        elif location == 'destination':
            speak('Provide destination location:')
        else:
            return ""

        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        recognized_text = recognizer.recognize_google(audio).lower()
        speak(f"Recognized: {recognized_text}")
        return recognized_text
    except sr.UnknownValueError:
        speak("Sorry, I could not understand the audio.")
        return ""
    except sr.RequestError as e:
        speak(f"Could not request results from the speech recognition service; {e}")
        return ""

def load_data(file_path):
    df = pd.read_excel(file_path)
    
    print("Column names in the Excel file:", df.columns)

    
    df.columns = df.columns.str.strip().str.lower()

    df['source'] = df['source'].str.lower().str.strip()
    df['destination'] = df['destination'].str.lower().str.strip()
    df['stops'] = df['stops'].apply(lambda x: [stop.lower().strip() for stop in x.split(',')])  
    
    return df

def find_all_routes(df, current_location, destination):
    graph = {}
    for _, row in df.iterrows():
        stops = row['stops']
        for i, stop in enumerate(stops):
            if stop not in graph:
                graph[stop] = set()
            if i > 0:
                graph[stop].add(stops[i-1])
            if i < len(stops) - 1:
                graph[stop].add(stops[i+1])
    queue = deque([(current_location, [current_location])])
    all_routes = []

    while queue:
        current_stop, path = queue.popleft()

        if current_stop == destination:
            all_routes.append(path)
            continue

        if current_stop not in graph:
            continue 

        for next_stop in graph[current_stop]:
            if next_stop not in path:
                queue.append((next_stop, path + [next_stop]))

    return all_routes

def get_routes_using_buses(df, path):
    routes = []
    for i in range(len(path) - 1):
        for _, row in df.iterrows():
            stops = row['stops']
            
            if path[i] in stops and path[i+1] in stops:
                routes.append(row['bus number'])
                break
    return routes

def find_routes(file_path, current_location, destination):
    df = load_data(file_path)
    all_paths = find_all_routes(df, current_location, destination)

    if not all_paths:
        speak("No routes found from your current location to your destination.")
        return "No routes found from your current location to your destination."

    result = "Possible routes:\n"
    for path in all_paths:
        bus_routes = get_routes_using_buses(df, path)
        route_info = f"Path: {' -> '.join(path)}\nBuses: {', '.join(bus_routes)}\n\n"
        speak(route_info)
        result += route_info

    return result

def udp_listener(host='0.0.0.0', port=12345):
    """Function to listen for UDP messages from ESP32S3 Sense."""
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind((host, port))
    print(f"Listening for UDP messages on {host}:{port}")

    while True:
        data, addr = sock.recvfrom(1024)  
        message = data.decode().strip()
        print(f"Received message: '{message}' from {addr}")

        if message == 'busDetected':
            speak("Bus arrived. Get into the bus")

udp_thread = threading.Thread(target=udp_listener)
udp_thread.daemon = True
udp_thread.start()

file_path = "/root/csvfiles/BusRouteData.xlsx"
current_location = Recognize_Speech('current')
destination = Recognize_Speech('destination')

routes = find_routes(file_path, current_location, destination)
print(routes)
detect_motion()

: 